In [1]:
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from time import sleep
from lxml import etree
import pandas as pd
from selenium.webdriver.common.by import By  
from selenium.webdriver import ActionChains
import  requests


In [2]:
#cd C:\Users\yuhuyi\AppData\Local\Google\Chrome\Application\
# chrome.exe --remote-debugging-port=9222 --user-data-dir="C:\selenium\ChromeProfile"
# https://compass.jinritemai.com/login

chrome_options = Options()
chrome_options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")
bro = webdriver.Chrome(options=chrome_options)
window_handles = bro.window_handles
bro.switch_to.window(window_handles[0])
bro.execute_script("window.scrollTo(0,document.body.scrollHeight)")
bro.maximize_window()

In [3]:
# def File_merge_json(Folder):  #Folder文件夹路径
#     try:
#         File_List = []  #存储文件夹下所有文件的路径
#         DataFrame_List = []  #存储dataframe
#         for root,dirs,files in os.walk(Folder):
#             for file in files:
#                 file_with_path = os.path.join(root,file)
#                 File_List.append(file_with_path)
#                 data_original = pd.read_json(file_with_path,encoding="utf-8")
#                 DataFrame_List.append(data_original)
#         datas = DataFrame_List
#         return datas
#     except  Exception as e: 
#         print(e)
# dataAll = File_merge_json(r"G:\GitWorkPlace\WorkPlace\PythonWorkPlace\spider\spidertest\抖音电商罗盘spider\抖店爬虫\电信100")
# area=pd.read_excel(r"./省份.xlsx")



# need_data2 = pd.DataFrame(columns=["店铺名称","成交金额_min","成交金额_max","点击次数_min","点击次数_max","商品ID","商品名称","是否首次上榜","价格区间","排名"])
# for a in dataAll:
#     datasss = pd.DataFrame(columns=["店铺名称","成交金额_min","成交金额_max","点击次数_min","点击次数_max","商品ID","商品名称","是否首次上榜","价格区间","排名"])
#     lsits = a["data"][0]
#     for i in lsits:
#         info_dicts = {}
#         info_dicts["店铺名称"] = i["product_info"]["shop_list"][0]["shop_name"]
#         info_dicts["商品名称"] = i["product_info"]["name"]
#         info_dicts["商品ID"] = i["product_info"]["id"]
#         # info_dicts["排名变化"] = i["product_info"]["rank_change"]
#         try:
#             info_dicts["点击次数_min"] = i["product_click_cnt"]["value_range"][0]["value"]
#             info_dicts["点击次数_max"] = i["product_click_cnt"]["value_range"][1]["value"]
#         except Exception as e:
#             # print(e)
#             pass
#         finally:
#             try:
#                 info_dicts["成交金额_min"] = i["new_pay_amt"]["value_range"][0]["value"]
#                 info_dicts["成交金额_max"] = i["new_pay_amt"]["value_range"][1]["value"]
#             except Exception as e:
#                 # print(e)
#                 pass   
#             finally:    
#                 info_dicts["价格区间"] = i["product_info"]["price_bin"]
#                 info_dicts["是否首次上榜"] = i["product_info"]["newly_on_ranking"]
#                 info_dicts["排名"] = i["product_info"]["rank"]
#                 info_df = pd.DataFrame(info_dicts,index=[0])
#                 datasss = pd.concat([datasss,info_df])
#     need_data2 = pd.concat([need_data2,datasss])
    


In [4]:
#获取下一页的元素
A = bro.find_element(By.XPATH,'//button[@class="ecom-pagination-item-link"]')

In [5]:
def get_data(html_str):
    HtmlData = etree.HTML(html_str)
    data = pd.DataFrame(columns=["店铺名称","成交金额","点击次数","商品名称","价格区间","排名"])
    tr_list = HtmlData.xpath("//tr[@class='ecom-table-row ecom-table-row-level-0']")
    for tr in tr_list:  
        info_dicts = {}
        info_dicts["店铺名称"] = tr.xpath(".//div[@class='name-qfS2bZ']/text()")[0]
        info_dicts["成交金额"] = tr.xpath(".//div[@class='numText-qvEDO1']/text()")[0]
        info_dicts["点击次数"] = tr.xpath(".//div[@class='numText-qvEDO1']/text()")[1]
        info_dicts["商品名称"] = tr.xpath(".//div[@class='name-olPpkd']/text()")[0]
        info_dicts["排名"] = tr.xpath(".//div[@class='rank-uKOWOn']/text()")[0]
        info_dicts["价格区间"] = tr.xpath(".//div[@class='price-T7iKi4']/span/text()")[0]
        info_df = pd.DataFrame(info_dicts,index=[0])
        data = pd.concat([data,info_df])
    return data

In [ ]:
need_data = pd.DataFrame(columns=["店铺名称","成交金额","点击次数","商品名称","价格区间","排名"])
i = 1
while i < 20:
    print(i)
    page = bro.page_source
    sleep(1)
    datasss=get_data(page)
    sleep(1)
    need_data = pd.concat([need_data,datasss])
    bro.execute_script("arguments[0].click()", A)
    i=i+1
    
else:
    page = bro.page_source
    sleep(1)
    datasss=get_data(page)
    sleep(1)
    need_data = pd.concat([need_data,datasss])
    pass

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [7]:
pd.set_option('display.max_rows', None)
need_data.drop_duplicates(inplace=True)
need_data

,店铺名称,成交金额,点击次数,商品名称,价格区间,排名
0,苏小西生活充值服务店,¥250-¥500,0-50,广东移动分省话费充值中心快充100元,¥95-¥100,191
0,达达通信,¥250-¥500,---,江西移动话费充值100元（0-3小时到账）B,¥97-¥100,192
0,云山生活充值店,¥250-¥500,---,【新1】河南移动 100元话费充值W,¥97-¥97,193
0,云山生活充值店,¥250-¥500,---,【新1】陕西移动 100元话费充值W,¥97-¥97,194
0,闪云通信专营店,¥250-¥500,50-100,全国移动 100元话费 3小时内到账,¥101-¥101,195
0,向上生活充值专营店,¥250-¥500,---,陕西移动100元话费充值（zc1）,¥100-¥100,196
0,宽厚话费充值专营店,¥250-¥500,0-50,【充值中心CZ】吉林移动话费快充100元,¥100-¥100,197
0,腾嘉话费充值专营店,¥250-¥500,0-50,[充值中心] 山东移动话费充值100元(0-3小时充值到,¥100-¥100,198
0,达达通信,¥250-¥500,0-50,河北移动话费快充100元（0-3小时到账）P,¥100-¥100,199
0,智信话费充值专营店,¥250-¥500,---,【直充1】黑龙江移动话费100元,¥97-¥100,200


In [ ]:
bro.page_source

In [ ]:
# C = bro.find_element(By.XPATH,'//span[@class="anticon anticon-left"]')
# bro.execute_script("arguments[0].click()", C)

In [ ]:
need_data.drop_duplicates(inplace=True)

In [ ]:
need_data.to_excel(".\spider数据.xlsx",index=False)